In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import numpy as np
import transformers

In [2]:
from transformers import DistilBertTokenizerFast,DistilBertForSequenceClassification

In [3]:
model_checkpoint='distilbert-base-uncased'
tokenizer=DistilBertTokenizerFast.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [4]:
model=DistilBertForSequenceClassification.from_pretrained(model_checkpoint,id2label={0:"NEG", 1:"POS"}, label2id={"NEG":0, "POS":1})

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
config=

AttributeError: 'DistilBertForSequenceClassification' object has no attribute 'summary'

In [5]:
from datasets import load_dataset

In [7]:
imdb_train=load_dataset('imdb',split='train')
imdb_test=load_dataset('imdb',split="test[:6250]+test[-6250:]")

In [8]:
imdb_val=load_dataset('imdb', split="test[6250:12500]+test[-12500:-6250]")

In [11]:
imdb_train.shape,imdb_test.shape,imdb_val.shape

((25000, 2), (12500, 2), (12500, 2))

# taking small dataset

In [12]:
imdb_train= load_dataset('imdb', split="train[:2000]+train[-2000:]")
imdb_test= load_dataset('imdb', split="test[:500]+test[-500:]")
imdb_val= load_dataset('imdb', split="test[500:1000]+test[-1000:-500]")

In [24]:
enc_train = imdb_train.map(lambda e: tokenizer( e['text'], padding=True,
truncation=True), batched=True, batch_size=1000)
enc_test = imdb_test.map(lambda e: tokenizer( e['text'], padding=True,
truncation=True), batched=True, batch_size=1000)
enc_val = imdb_val.map(lambda e: tokenizer( e['text'], padding=True,
truncation=True), batched=True, batch_size=1000)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [36]:
from transformers import TrainingArguments,Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


In [18]:
training_args=TrainingArguments(
    output_dir='training',
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy='steps',
    logging_dir='./logs',
    logging_steps=200,
    evaluation_strategy= 'steps',
    
)

In [38]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    Precision, Recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
    'Accuracy': acc,
    'F1': f1,
    'Precision': Precision,
    'Recall': Recall
}

In [39]:
trainer = Trainer(
model=model,
args=training_args,
train_dataset=enc_train,
eval_dataset=enc_val,
compute_metrics= compute_metrics
)


In [40]:
history=trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
200,0.019300,0.873880,0.868000,0.867614,0.872343,0.868000
